In [ ]:
import pandas as pd

import json

from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk, streaming_bulk


import sys
from datetime import datetime

In [ ]:
# https://kibana.kibio.science/app/discover#/?_g=(filters:!(),refreshInterval:(pause:!t,value:0),time:(from:now-15m,to:now))&_a=(columns:!(),filters:!(),index:'6527d100-13e9-11ee-a697-0b0e571db997',interval:auto,query:(language:kuery,query:''),sort:!())


In [2]:
es_out = Elasticsearch("http://192.168.3.76:9201", basic_auth=('elastic','ZqDNgKrksnLRkTRASz6Y'),verify_certs=False,ssl_show_warn=False)
#es_out = Elasticsearch("http://192.168.3.76:9201", basic_auth=('elastic',''),verify_certs=False,ssl_show_warn=False)
print(es_out)

<Elasticsearch(['http://192.168.3.76:9201'])>


In [3]:
from intermine.webservice import Service


In [9]:
def intermine2es(service_taxon, taxon_id, class_name, field_list, start_pos, end_pos):
    
    service = Service("https://www."+service_taxon+".org/"+service_taxon+"/service")
    #service = Service("https://www.flymine.org/flymine/service")

    # Get a new query on the class (table) you will be querying:
    query = service.new_query(class_name)

    #query.add_view("diseaseId", "diseaseType", "name", "primaryIdentifier")
    for field_name in field_list:
        print(field_name)
        query.add_view(field_name)

    ctr = 0
    res = ''
    jsonStr = ''

    for row in query.rows():
        ctr += 1
        if ctr % 1000 == 0:
            print(ctr, res)
            print(jsonStr)

        #print(row["diseaseId"], row["diseaseType"], row["name"], row["primaryIdentifier"], row["alleles.primaryIdentifier"], row["genes.primaryIdentifier"])
        row_dict = row.__dict__
        #print(row_dict)

        if ctr < start_pos:
            continue
        if ctr > end_pos:
            break

        jsonDoc = {}
        jsonDoc['@type'] = 'intermine:Relation'
        jsonDoc['taxonomy'] = 'taxon:'+taxon_id

        field_list2 = []
        for property in field_list:
            field_list2.append(property.split('.')[0])
        field_list2.append('id')

        jsonDoc['predicate'] = field_list2[0]

        #print(field_list2)

        ctri = 0
        for name in row_dict['views']:
            property = name.split('.')[1]
            if property in field_list2 :
                property = property.replace('id','intermine_id')
                #print(property, row_dict['data'][ctri])
                jsonDoc[property] = row_dict['data'][ctri]
            if property == field_list2[0] :
                jsonDoc['uri'] = row_dict['data'][ctri]
                
                
            ctri += 1

        jsonDoc['@id'] = 'intermine:'+class_name+':' + str(jsonDoc['intermine_id'])
        jsonDoc['class_name'] = class_name
        doc_id = 'intermine:'+class_name+':' + str(jsonDoc['intermine_id']) + '_' + str(ctr)

        jsonStr = json.dumps(jsonDoc)

        #print(jsonStr)

        #res = es_out.index(index='intermine_relation', id=doc_id, document=jsonStr)

#'''
        yield {
            "_index": 'intermine_relation',
            "_id": doc_id,
            "_source": jsonStr
            }
#'''
    
    #print(ctr, res)


bulk(es_out, intermine2es('humanmine','9606','Gene',["diseases.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)
bulk(es_out, intermine2es('humanmine','9606','Gene',["proteins.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)
bulk(es_out, intermine2es('humanmine','9606','Disease',["genes.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)
bulk(es_out, intermine2es('humanmine','9606','Protein',["genes.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)

bulk(es_out, intermine2es('flymine','7227','Gene',["diseases.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)
bulk(es_out, intermine2es('flymine','7227','Gene',["proteins.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)
bulk(es_out, intermine2es('flymine','7227','Disease',["genes.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)
bulk(es_out, intermine2es('flymine','7227','Protein',["genes.primaryIdentifier"], 0, 1000000), chunk_size=1000, max_retries=20, initial_backoff=5, max_backoff=5)



diseases.primaryIdentifier
1000 
{"@type": "intermine:Relation", "taxonomy": "taxon:9606", "predicate": "diseases", "intermine_id": 1015926, "diseases": "OMIM:249100", "uri": "OMIM:249100", "@id": "intermine:Gene:1015926", "class_name": "Gene"}
2000 
{"@type": "intermine:Relation", "taxonomy": "taxon:9606", "predicate": "diseases", "intermine_id": 1062644, "diseases": "OMIM:619361", "uri": "OMIM:619361", "@id": "intermine:Gene:1062644", "class_name": "Gene"}
3000 
{"@type": "intermine:Relation", "taxonomy": "taxon:9606", "predicate": "diseases", "intermine_id": 1091786, "diseases": "OMIM:604765", "uri": "OMIM:604765", "@id": "intermine:Gene:1091786", "class_name": "Gene"}
4000 
{"@type": "intermine:Relation", "taxonomy": "taxon:9606", "predicate": "diseases", "intermine_id": 1082164, "diseases": "OMIM:614889", "uri": "OMIM:614889", "@id": "intermine:Gene:1082164", "class_name": "Gene"}
5000 
{"@type": "intermine:Relation", "taxonomy": "taxon:9606", "predicate": "diseases", "intermine_i

(204083, [])